# Plasmid: A python based tool for gene editing
This notebook illustrates the various functions contained in the `plasmid` class, which can be used to read and manipulate genbank files.

## Table of Contents
1. [Reading and viewing genbank files](#reading_genbank)
    1. [Visualizing genbank records](#visualizing_genbank)
    1. [Selecting and filtering for genomic features](#filtering_genbank)   
    1. [Gene translation to amino acids](#gene_translation)   
1. [Creating new genbank records](#gene_concat)
    1. [Writing genbank files](#genbank_write)
    1. [Sequence annotation](#gene_annotation)
    1. [Searching for open reading frames](#gene_ORF_search)

In [3]:
import plasmid as pge
import importlib
importlib.reload(pge)

<module 'plasmid' from '/home/zchen/Public/python/lib/python3.11/site-packages/plasmid/__init__.py'>

## Reading genbank or fasta files <a class="anchor" id="reading_genbank"></a>

Genbank, fasta, and text files can be read using `read_genbank`. This function loads genbank information via `biopython` into a pandas like dataframe called `Plasmid`. This is analogous to `read_csv` in pandas. The underlying data in Biopython can be sliced, sorted, filtered, and concatenated like a string.

In [4]:
help(pge.read_genbank)

Help on function read_genbank in module plasmid.plasmid:

read_genbank(fname)
    Reads a genbank file into plasmid class object
    fname = genbank file to read
    returns Plasmid object with data loaded



In [5]:
df = pge.read_genbank('../data/xRFP.gb')
df

reading  ../data/xRFP.gb  as genbank file


<class 'plasmid.plasmid.Plasmid'> at 0x7f443caa33d0
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
                              locus_tag          type        location  length   
0                         AmpR promoter      promoter       [0:29](+)      29  \
1                              AmpR RBS           RBS      [29:70](+)      41   
2                          SpectomycinR           CDS     [70:868](+)     798   
3                             BBa_B0053    terminator    [878:948](+)      70   
4   J23108, 0.51, Constitutive Promoter      promoter  [1032:1067](+)      35   
5          RBS 1.00 strength, BBa_B0034           RBS  [1086:1098](+)      12   
6                       LacI, BBa_C0012           CDS  [1104:2196](+)    1092   
7                             Esp3I_fix  misc_feature  [2072:2078](+)       6   
8         Forward Terminator, BBa_B

### Visualizing the genbank record <a class="anchor" id="visualizing_genbank"></a>

The raw sequence with color annotations can be displayed with `.get_colored()`, which generates a string containing colorization flags that can be interpreted by a bash terminal. The dataframe can also be sliced or cast as a string to obtain the raw sequence.

In [6]:
help(df.get_colored)

Help on method get_colored in module plasmid.plasmid:

get_colored(width=0) method of plasmid.plasmid.Plasmid instance
    Print out colored representation of Plasmid
    width = number of characters per line



In [7]:
# displays the first 1000bp of the plasmid
x = df[0:1000]
print(x.__repr__())
out = x.get_colored()
print(out)

<class 'plasmid.plasmid.Plasmid'> at 0x7f4455a3f950
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
                   locus_tag        type      location  length    color
0  AmpR promoter subsequence    promoter     [0:29](+)      29  #0f7ffe
1       AmpR RBS subsequence         RBS    [29:70](+)      41     cyan
2   SpectomycinR subsequence         CDS   [70:868](+)     798  #d40037
3      BBa_B0053 subsequence  terminator  [878:948](+)      70  #9d1b1c
total length:1000

TTCAAATATCTATCCGCTCATGAGACAATAACCCTGATAAATGCTTCAATAATATTGAAAAAGGAAGAATATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAGCACTCAACGTCATCGGGCGCCACTTGGAGTCGACGTTGCTGGCCGTGCATTTGTACGGCTCCGCACTGGATGGCGGATTGAAACCGTACAGTGATATTGATTTGCTGGTGACTGTAGCTGCACCGCTCAATGATGCCGTGCGGCAAGCCCTGCTCGTCGATCTCTTGGAGGTTTCAGCTTCCCCTGGCCAAAACAAGGCACTCCGCGCCTTGGAAGTGACCATCGTCGTGCACAGTGACATCGTACCTTGGCGTTATCCGGCCAGGCGG

### Selecting and filtering for genomic features  <a class="anchor" id="filtering_genbank"></a>

Genes can also be selected and sliced out of the dataframe via boolean selection like in pandas.

In [8]:
help(df.__getitem__)

Help on method __getitem__ in module plasmid.plasmid:

__getitem__(key) method of plasmid.plasmid.Plasmid instance
    Splice Plasmid sequence with range or filter through features on the Plasmid
    key = if key is a range or slice, then slice using base pair index from start to end
          if key is a str, then return the column values of pandas dataframe
          if key is an int, then return feature at the feature index
          if key is a list of integers, then return features only at the indexes in the list 
          if key is a boolean array, then return features only at indexes which evaluate True



In [9]:
# select all genes which are terminator sequences
x = df[800:2500]
y = x[x['type']=='terminator']
print(y.__repr__())
print(y.get_colored())

<class 'plasmid.plasmid.Plasmid'> at 0x7f4455b62ad0
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
                                   locus_tag        type        location   
0                      BBa_B0053 subsequence  terminator     [78:148](+)  \
1  Forward Terminator, BBa_B1002 subsequence  terminator  [1396:1430](+)   
2             S. pyog terminator subsequence  terminator  [1530:1571](+)   
3  Forward Terminator, BBa_B1010 subsequence  terminator  [1571:1611](+)   
4                  T0 terminator subsequence  terminator  [1635:1700](+)   

   length    color  
0      70  #9d1b1c  
1      34  #804000  
2      41  #804000  
3      40  #804000  
4      65  #804000  
total length:1700

ATCAGGTGGCGGCGCTCATTAAATTCGTGAAGTATGAAGCAGTTAAACTGCTTGGTGCCAGCCAATGATAATACTAGCTCCGGCAAAAAAACGGGCAAGGTGTCACCACCCTGCCCTTTTTCTTTAAAACCGAAAAGATTACTTCGCGAAATAGGCGTATCA

### Filtering by keyword <a class="anchor" id="filtering_keyword"></a>
Keyword selection is also possible because of the compatibility with pandas. In the following, we create a boolean array by looking for all strings in the `locus_tag` column which contains `LacI`. We use the boolean array to filter for genomic features satisfying this keyword. `.splice()` is used to slice out the gene sequence from the parent plasmid.

In [10]:
help(df.splice)

Help on method splice in module plasmid.plasmid:

splice(inplace=False) method of plasmid.plasmid.Plasmid instance
    Get a slice sequence from start of all features to end of all features



In [11]:
# Select all genes containing LacI in the locus_tag column
filt = df['locus_tag'].str.contains('LacI')
y = df[filt]
y = y.splice()
print(y.__repr__())
print(y.get_colored())

<class 'plasmid.plasmid.Plasmid'> at 0x7f443dba6590
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
                     locus_tag type     location  length    color
0  LacI, BBa_C0012 subsequence  CDS  [0:1092](+)    1092  #fc6665
total length:1092

ATGGTGAATGTGAAACCAGTAACGTTATACGATGTCGCAGAGTATGCCGGTGTCTCTTATCAGACCGTTTCCCGCGTGGTGAACCAGGCCAGCCACGTTTCTGCGAAAACGCGGGAAAAAGTGGAAGCGGCGATGGCGGAGCTGAATTACATTCCCAACCGCGTGGCACAACAACTGGCGGGCAAACAGTCGTTGCTGATTGGCGTTGCCACCTCCAGTCTGGCCCTGCACGCGCCGTCGCAAATTGTCGCGGCGATTAAATCTCGCGCCGATCAACTGGGTGCCAGCGTGGTGGTGTCGATGGTAGAACGAAGCGGCGTCGAAGCCTGTAAAGCGGCGGTGCACAATCTTCTCGCGCAACGCGTCAGTGGGCTGATCATTAACTATCCGCTGGATGACCAGGATGCCATTGCTGTGGAAGCTGCCTGCACTAATGTTCCGGCGTTATTTCTTGATGTCTCTGACCAGACACCCATCAACAGTATTATTTTCTCCCATGAAGACGGTACGCGACTGGGCGTGGAGCATCTGGTCGCATTGGGTCACCAGCAAATCGCGCTGTTAGCGGGCCCATTAAGTTCTGTCTCGGCGCGTCTGCGTCTGGCTGGCTGGC

### Sequence translation  <a class="anchor" id="gene_translation"></a>

DNA sequences can be translated to amino acid sequences via `.translate()`

In [12]:
help(df.translate)

Help on method translate in module plasmid.plasmid:

translate(strand=None, frame=0, table='Standard') method of plasmid.plasmid.Plasmid instance
    Translate the DNA sequence to amino acids
    frame = codon reading frame such as 0, 1, or 2
    table = codon table to use. Standard codon table is used by default
    return a string



In [13]:
y = x[x['locus_tag'].str.contains('LacI')].splice()
print('This is the DNA sequence')
print(y.get_colored())
print('This is the amino acid sequence')
print(y.translate())

This is the DNA sequence
ATGGTGAATGTGAAACCAGTAACGTTATACGATGTCGCAGAGTATGCCGGTGTCTCTTATCAGACCGTTTCCCGCGTGGTGAACCAGGCCAGCCACGTTTCTGCGAAAACGCGGGAAAAAGTGGAAGCGGCGATGGCGGAGCTGAATTACATTCCCAACCGCGTGGCACAACAACTGGCGGGCAAACAGTCGTTGCTGATTGGCGTTGCCACCTCCAGTCTGGCCCTGCACGCGCCGTCGCAAATTGTCGCGGCGATTAAATCTCGCGCCGATCAACTGGGTGCCAGCGTGGTGGTGTCGATGGTAGAACGAAGCGGCGTCGAAGCCTGTAAAGCGGCGGTGCACAATCTTCTCGCGCAACGCGTCAGTGGGCTGATCATTAACTATCCGCTGGATGACCAGGATGCCATTGCTGTGGAAGCTGCCTGCACTAATGTTCCGGCGTTATTTCTTGATGTCTCTGACCAGACACCCATCAACAGTATTATTTTCTCCCATGAAGACGGTACGCGACTGGGCGTGGAGCATCTGGTCGCATTGGGTCACCAGCAAATCGCGCTGTTAGCGGGCCCATTAAGTTCTGTCTCGGCGCGTCTGCGTCTGGCTGGCTGGCATAAATATCTCACTCGCAATCAAATTCAGCCGATAGCGGAACGGGAAGGCGACTGGAGTGCCATGTCCGGTTTTCAACAAACCATGCAAATGCTGAATGAGGGCATCGTTCCCACTGCGATGCTGGTTGCCAACGATCAGATGGCGCTGGGCGCAATGCGCGCCATTACCGAGTCCGGGCTGCGCGTTGGTGCGGATATCTCGGTAGTGGGATACGACGATACCGAAGACAGCTCATGTTATATCCCGCCGTTAACCACCATCAAACAGGATTTTCGCCTGCTGGGGCAAACCAGCGTGGACCGCTTGCTGCAACTCTCTCAGGGCCAGGCGGTGAAGGGCAATCAGCTGTTGCCCGTTTCA

## Creating new genbank records <a id='gene_concat'></a>
### Concatenating genomic features
New genes and constructs can be generated by combining genetic parts via concatenation. The following extracts the RFP fluorescent reporter, ribosome binding site, and pLac promoter genes from existing genbank records and concatenates the components to form a new genbank record.

In [14]:
# slice out the RFP gene
RFP = pge.read_genbank('../data/dcas9_RFP.gb')
RFP = RFP[RFP['locus_tag'].str.contains('mRFP')].splice()
# slice out the ribosome binding site
RBS = pge.read_genbank('../data/xRFP.gb')
RBS = RBS[RBS['locus_tag'].str.contains('BBa_B0034')].splice()
# slice out the promoter
pLac = pge.read_genbank('../data/xRFP.gb')
pLac = pLac[pLac['locus_tag'].str.contains('pLac')].splice()

# assemble the promoter, rbs, and mRFP, adds spacer sequences between promoter, rbs, and rfp
df = pLac + 'gagacc' + RBS + 'ggtctc' + RFP
print(df.__repr__())
print(df.get_colored())

reading  ../data/dcas9_RFP.gb  as genbank file
reading  ../data/xRFP.gb  as genbank file
reading  ../data/xRFP.gb  as genbank file
<class 'plasmid.plasmid.Plasmid'> at 0x7f44941fc350
molecule_type:DNA
topology:circular
                                         locus_tag      type     location   
0  pLacO2, single operon pLac ZC082818 subsequence  promoter    [0:38](+)  \
1         RBS 1.00 strength, BBa_B0034 subsequence       RBS   [44:56](+)   
2      mRFP, uniprot drFP583, pdb 2H5O subsequence       CDS  [62:740](+)   

   length    color  
0      38  #008080  
1      12     cyan  
2     678  #800040  
total length:740

AATTGACAATGTGAGCGAGTAACAAGATACTGAGCACAgagaccAAAGAGGAGAAAggtctcATGGCGAGTAGCGAAGACGTTATCAAAGAGTTCATGCGTTTCAAAGTTCGTATGGAAGGTTCCGTTAACGGTCACGAGTTCGAAATCGAAGGTGAAGGTGAAGGTCGTCCGTACGAAGGTACCCAGACCGCTAAACTGAAAGTTACCAAAGGTGGTCCGCTGCCGTTCGCTTGGGACATCCTGTCCCCGCAGTTCCAGTACGGTTCCAAAGCTTACGTTAAACACCCGGCTGACATCCCGGACTACCTGAAACTGTCCTTCCCGGAAGGTTTCAAATGGGAACGTGTTATGAACTTCGAAGACGGTGG

### Writing genbank records  <a class="anchor" id="genbank_write"></a>
We can write new gene products to genbank records with the `.to_genbank` function.

In [15]:
help(df.to_genbank)

Help on method to_genbank in module plasmid.plasmid:

to_genbank(filename, timestamp=False) method of plasmid.plasmid.Plasmid instance
    Writes the Plasmid SeqRecord to a filename
    filename = filename of output file
    timestamp = adds a timestamp to the output file



In [16]:
# create a new gene
df = pLac + 'gagacc' + RBS + 'ggtctc' + RFP
print(df.__repr__())
print(df.get_colored())
# write the gene to genbank
df.to_genbank('test.gb')
# read the raw genbank file
with open('test.gb','r') as f:
    print(f.read())

<class 'plasmid.plasmid.Plasmid'> at 0x7f443caf8750
molecule_type:DNA
topology:circular
                                         locus_tag      type     location   
0  pLacO2, single operon pLac ZC082818 subsequence  promoter    [0:38](+)  \
1         RBS 1.00 strength, BBa_B0034 subsequence       RBS   [44:56](+)   
2      mRFP, uniprot drFP583, pdb 2H5O subsequence       CDS  [62:740](+)   

   length    color  
0      38  #008080  
1      12     cyan  
2     678  #800040  
total length:740

AATTGACAATGTGAGCGAGTAACAAGATACTGAGCACAgagaccAAAGAGGAGAAAggtctcATGGCGAGTAGCGAAGACGTTATCAAAGAGTTCATGCGTTTCAAAGTTCGTATGGAAGGTTCCGTTAACGGTCACGAGTTCGAAATCGAAGGTGAAGGTGAAGGTCGTCCGTACGAAGGTACCCAGACCGCTAAACTGAAAGTTACCAAAGGTGGTCCGCTGCCGTTCGCTTGGGACATCCTGTCCCCGCAGTTCCAGTACGGTTCCAAAGCTTACGTTAAACACCCGGCTGACATCCCGGACTACCTGAAACTGTCCTTCCCGGAAGGTTTCAAATGGGAACGTGTTATGAACTTCGAAGACGGTGGTGTTGTTACCGTTACCCAGGACTCCTCCCTGCAAGACGGTGAGTTCATCTACAAAGTTAAACTGCGTGGTACCAACTTCCCGTCCGACGGTCCGGTTATGCAGAAAAAAACCATGGGTTGGGAAGCTTCCA

### Adding new annotations to genbank records  <a class="anchor" id="gene_annotation"></a>
Annotation of new genes or features can be done with the `.annotate` function. In the following, we annotate BsaI restriction enzyme sites in a new construct we just generated.

In [17]:
help(df.annotate)

Help on method annotate in module plasmid.plasmid:

annotate(name, sequence, feature='unknown', color=None, circular=True, inplace=False) method of plasmid.plasmid.Plasmid instance
    Adds annotations to a plasmid using sequence information
    name = name of the gene
    sequence = DNA or amino acid sequence of the feature
    feature = type of genetic feature such as cds, mRNA, primer_bind
    color = [fwd_color, rev_color] to use
    circular = search as though sequence is a circular construct
    inplace = performs modifications inplace
    returns a modified plasmid dataframe



In [18]:
# assemble the promoter, rbs, and mRFP, adds spacer sequences between promoter, rbs, and rfp
df = pLac + 'gagacc' + RBS + 'ggtctc' + RFP
# annotate BsaI sites
df = df.annotate(name='BsaI', sequence='GGTCTC', color=['red','orange'], feature='protein_bind')
print(df.__repr__())
print(df.get_colored())

<class 'plasmid.plasmid.Plasmid'> at 0x7f443caba710
molecule_type:DNA
topology:circular
                                         locus_tag          type     location   
0  pLacO2, single operon pLac ZC082818 subsequence      promoter    [0:38](+)  \
1                                             BsaI  protein_bind   [38:44](-)   
2         RBS 1.00 strength, BBa_B0034 subsequence           RBS   [44:56](+)   
3                                             BsaI  protein_bind   [56:62](+)   
4      mRFP, uniprot drFP583, pdb 2H5O subsequence           CDS  [62:740](+)   

   length    color  
0      38  #008080  
1       6   orange  
2      12     cyan  
3       6      red  
4     678  #800040  
total length:740

AATTGACAATGTGAGCGAGTAACAAGATACTGAGCACAgagaccAAAGAGGAGAAAggtctcATGGCGAGTAGCGAAGACGTTATCAAAGAGTTCATGCGTTTCAAAGTTCGTATGGAAGGTTCCGTTAACGGTCACGAGTTCGAAATCGAAGGTGAAGGTGAAGGTCGTCCGTACGAAGGTACCCAGACCGCTAAACTGAAAGTTACCAAAGGTGGTCCGCTGCCGTTCGCTTGGGACATCCTGTCCCCGCAGTTCCAGTACGGTTCCAAAGCTTACGTT

### Annotation via amino acid search
Annotations can also be done via searching for amino acid sequences. In the following, we search for a short amino acid sequence in the LacI gene and annotate it. Exact amino acid search is performed in both forward and reverse orientations for the given gene.

In [19]:
print(RFP.translate())
df = pLac + 'gagacc' + RBS + 'ggtctc' + RFP
aaseq = 'DGALKGEIKMRLKLKDG'
df = df.annotate(name='peptide', sequence=aaseq, color='orange')
df = df.drop_duplicates()
print(df.get_colored())

MASSEDVIKEFMRFKVRMEGSVNGHEFEIEGEGEGRPYEGTQTAKLKVTKGGPLPFAWDILSPQFQYGSKAYVKHPADIPDYLKLSFPEGFKWERVMNFEDGGVVTVTQDSSLQDGEFIYKVKLRGTNFPSDGPVMQKKTMGWEASTERMYPEDGALKGEIKMRLKLKDGGHYDAEVKTTYMAKKPVQLPGAYKTDIKLDITSHNEDYTIVEQYERAEGRHSTGA*
AATTGACAATGTGAGCGAGTAACAAGATACTGAGCACAgagaccAAAGAGGAGAAAggtctcATGGCGAGTAGCGAAGACGTTATCAAAGAGTTCATGCGTTTCAAAGTTCGTATGGAAGGTTCCGTTAACGGTCACGAGTTCGAAATCGAAGGTGAAGGTGAAGGTCGTCCGTACGAAGGTACCCAGACCGCTAAACTGAAAGTTACCAAAGGTGGTCCGCTGCCGTTCGCTTGGGACATCCTGTCCCCGCAGTTCCAGTACGGTTCCAAAGCTTACGTTAAACACCCGGCTGACATCCCGGACTACCTGAAACTGTCCTTCCCGGAAGGTTTCAAATGGGAACGTGTTATGAACTTCGAAGACGGTGGTGTTGTTACCGTTACCCAGGACTCCTCCCTGCAAGACGGTGAGTTCATCTACAAAGTTAAACTGCGTGGTACCAACTTCCCGTCCGACGGTCCGGTTATGCAGAAAAAAACCATGGGTTGGGAAGCTTCCACCGAACGTATGTACCCGGAAGACGGTGCTCTGAAAGGTGAAATCAAAATGCGTCTGAAACTGAAAGACGGTGGTCACTACGACGCTGAAGTTAAAACCACCTACATGGCTAAAAAACCGGTTCAGCTGCCGGGTGCTTACAAAACCGACATCAAACTGGACATCACCTCCCACAACGAAGACTACACCATCGTTGAACAGTACGAACGTGCTGAAGGTCGTCACTCCACCGGTGCTTAA


/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translatio

### Searching for open reading frames  <a class="anchor" id="gene_ORF_search"></a>
Genbank records may sometimes contain unannotated genes. In these situations, we can perform a search of open reading frames in the sequence to help identify hidden genomic features of plasmid. In the following, we will load an annotated genbank record of a plasmid and delete the antibiotic resistance gene. We can search for the open reading frames with `Aligner.search_ORF` to recover some missing annotations.

In [20]:
help(pge.Aligner.search_ORF)

Help on function search_ORF in module plasmid.aligner:

search_ORF(seq, table='Standard', fwd_only=False)
    Generates a dataframe of open reading frames for a given sequence
    seq = input sequence
    table = codon table to use
    return pandas Dataframe with columns
        [name, start, stop, orientation, amino acid sequence]



In [21]:
# load a genbank record and delete the antibiotic resistance gene
df = pge.read_genbank('../data/xRFP.gb')
print(df[2].__repr__())
del(df[2])
print(df[:1000].__repr__())
# generate list of open reading frames
seq = str(df[:1000])
x = pge.Aligner.search_ORF(seq)
# compute length of the sequence
x['length'] = x['stop'] - x['start']
# get the top 5 longest sequences
x = x.sort_values(by=['length','orientation','start']).iloc[::-1]
col = ['start','stop','length','sequence']
print(x[col].iloc[:5])

reading  ../data/xRFP.gb  as genbank file
<class 'plasmid.plasmid.Plasmid'> at 0x7f445487f3d0
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
      locus_tag type     location  length    color
0  SpectomycinR  CDS  [70:868](+)     798  #d40037
total length:4465

<class 'plasmid.plasmid.Plasmid'> at 0x7f443d447690
molecule_type:DNA
topology:circular
data_file_division:   
date:05-DEC-2022
accessions:['<unknown', 'id>']
keywords:['']
source:
organism:. .
taxonomy:[]
comment:
ApEinfo:methylated:1
                   locus_tag        type      location  length    color
0  AmpR promoter subsequence    promoter     [0:29](+)      29  #0f7ffe
1       AmpR RBS subsequence         RBS    [29:70](+)      41     cyan
2      BBa_B0053 subsequence  terminator  [878:948](+)      70  #9d1b1c
total length:1000

  start stop length                                       

/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/zchen/Public/python/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translatio

In the above, the longest sequence is likely the missing genes. We can search on NCBI blast to see if this matches any missing genes.